# MUSA 5090 Final Project

This file is used for data exploration and feature engineering in order to build a machine learning model that predicts the sale price of properties in the Philadelphia.

In [1]:
from google.colab import drive # do not include this import in the cloud version
import geopandas as gpd
import pandas as pd
import numpy as np
import statsmodels.api as sm

## Datasets

Firstly, we import the required libraries and load our dataset. The dataset, opa_properties, is avaialable on OpenDataPhilly.

In [2]:
# mount drive to access the dataset
# only for data exploration purpose
# do not inlcude when deploy to cloud
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# read in properties dataset
opa_dir = '/content/drive/MyDrive/musa-final-proj/opa_properties_public.csv'
properties = pd.read_csv(opa_dir)
properties.head(200)

<ipython-input-3-0149586cf5df>:3: DtypeWarning: Columns (4,16,25,27,48,76) have mixed types. Specify dtype option on import or set low_memory=False.
  properties = pd.read_csv(opa_dir)


,objectid,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,...,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape
0,445763190,2022-05-24 12:13:13-04:00,NaN,NEC DOVER ST,NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,172.0,...,NaN,I,NaN,NaN,19132.0,CMX1,1001321879,NaN,NaN,SRID=2272;POINT ( 2689039.51193106 252319.375...
1,445763191,2022-05-24 12:13:13-04:00,NaN,SEC 27TH ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,32.0,...,NaN,I,NaN,NaN,19146.0,RSA5,1001179696,NaN,NaN,SRID=2272;POINT ( 2686489.39071898 229330.622...
2,445763192,2022-05-24 12:13:13-04:00,NaN,NWC 16TH ST,NaN,T38,ROW B/OFF-STR 2 STY STONE,3,MIXED USE,373.0,...,NaN,I,1950.0,NaN,19145.0,CMX1,1001408261,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2690626.09996523 221499.086...
3,445763193,2022-05-24 12:13:13-04:00,NaN,"343'7 1/8"" W 31ST ST",NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,169.0,...,NaN,I,NaN,NaN,19132.0,RSA5,1001168670,NaN,NaN,SRID=2272;POINT ( 2687281.27004074 250386.258...
4,445763194,2022-05-24 12:13:13-04:00,NaN,197' W OF 19TH ST,NaN,ZL0,MISC FUNERAL HOME MASONRY,4,COMMERCIAL,134.0,...,NaN,A,1920.0,NaN,19130.0,RM1,1001494378,246,FUNERAL HOME,SRID=2272;POINT ( 2691553.59778215 240243.694...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,445763289,2022-05-24 12:13:13-04:00,NaN,"334'11 1/4 "" N",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,164.0,...,NaN,I,NaN,NaN,19133.0,CMX2,1001598005,NaN,NaN,SRID=2272;POINT ( 2698753.98725273 249011.820...
196,445763290,2022-05-24 12:13:13-04:00,NaN,290' W OF 31ST ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,151.0,...,NaN,I,NaN,NaN,19121.0,RSA5,1001408622,NaN,NaN,SRID=2272;POINT ( 2687105.63915840 248735.226...
197,445763291,2022-05-24 12:13:13-04:00,NaN,"74'7 3/4""N TIOGA",NaN,SC,VACANT LAND COMMER < ACRE,6,VACANT LAND,201.0,...,NaN,I,NaN,NaN,19140.0,CMX2,1001625620,NaN,NaN,SRID=2272;POINT ( 2694809.80616473 256111.430...
198,445763292,2022-05-24 12:13:13-04:00,NaN,SW COR 50TH ST,NaN,S30,ROW W/OFF STR 2 STY MASON,3,MIXED USE,104.0,...,NaN,I,1925.0,NaN,19139.0,RSA5,1001412740,820,ROW MIXED-COM/RES-BLT AS RES,SRID=2272;POINT ( 2677389.92105706 240959.320...


In [ ]:
# check attribtue name
properties.columns.tolist()

Upon checking the dataset, we found the following attributes from the opa properties  to be potentially useful to predicting property values. We listed them here:

- basements
- central air
- fireplaces
- garage type
- number of bedrooms
- number of bathrooms
- number of rooms
- number stories
- parcel number
- quality grade
- total area
- view
- year built

Based on our prior knowledge in house price and the given data set, we believe that the sale price of houses may be related to both the internal and geographical attributes of the houses. Internal attributes refer to factors such as the age of the house, the number of bedrooms and bathrooms, the presence of amenities, and the overall condition of the house.

Regarding internal attributes, we selected the following variables from: the year the house was built, the number of bedrooms, the number of bathrooms, the number of air conditioning units, basement condition, the number of fireplaces, garage condition, the number of stories, total area, livable area, heating condition, view category, quality grade and the type of building description. Other indicators related to house sale prices, such as topography, internal condition, and external condition, were not included in the scope of indicators due to their correlation with the selected indicators.

In [4]:
# some baseline data processing work
properties_mdl = properties[['basements',
                          'category_code',
                          'census_tract',
                          'central_air', #
                          'fireplaces', #
                          'garage_type', #
                          'market_value',
                          'number_of_bedrooms', #
                          'number_of_bathrooms', #
                          'number_of_rooms', #
                          'number_stories',#
                          'parcel_number',
                          'quality_grade', #
                          'sale_price',
                          'type_heater', #
                          'total_area', #
                          'total_livable_area', #
                          'view_type', #
                          'building_code_description_new', #
                          'zip_code',
                          'year_built']] #

## Feature Enginnering

We calculated the age of the houses by subtracting 2024 from the year they were built and used it as a new time measurement indicator. Since the number of bedrooms and bathrooms are correlated, we used the total number of rooms as a new indicator. Specific quantities or detailed conditions of air conditioning, basement, fireplace, garage, and heating did not have a significant impact individually, but their presence or absence had a significant impact on house prices. Therefore, we created new binary variables for each of them. Regarding the number of stories, we collapsed their categories into single, double, and multiple. For total area, we chose the larger of the total area and livable area as the measurement indicator and applied a logarithmic transformation to achieve a normal distribution. For the view type near the house, we summarized the categories into three classes: Scenic (including Cityscape/Skyline, Flowing Water, Park/Green Area), Typical, and Others (Urban). For quality and building type, we performed similar processing, combining indicators above “average” into a “good” category and those “average” or below into a “bad” category.

In [ ]:
# calculate age
properties_mdl['Age'] = 2024 - properties_mdl['year_built']

In [ ]:
# recode rooms and view variables
properties_mdl['numRooms'] = np.select(
    [(properties_mdl['number_of_bedrooms'].isna()) & (~properties_mdl['number_of_bathrooms'].isna()),
     (properties_mdl['number_of_bathrooms'].isna()) & (~properties_mdl['number_of_bedrooms'].isna()),
      (properties_mdl['number_of_bathrooms'].isna()) & (properties_mdl['number_of_bedrooms'].isna())],
    [properties_mdl['number_of_bathrooms'],
     properties_mdl['number_of_bedrooms'],
     0],
    default=properties_mdl['number_of_bedrooms'] + properties_mdl['number_of_bathrooms']
)


properties_mdl['view'] = np.select(
    [properties_mdl['view_type'].isin(['I', '0']) | properties_mdl['view_type'].isna(),
     properties_mdl['view_type'].isin(['A', 'B', 'C'])],
    ['Typical','Scenic'],
    default='Urban'
)

In [ ]:
# recode amenities variables
properties_mdl['hasAC'] = np.where(properties_mdl['central_air'].isin(['1', 'Y']), 'Y', 'N')
properties_mdl['hasBasement'] = np.where(properties_mdl['basements'].isin(['1', '4', 'A', 'B', 'C', 'D', 'E', 'F']), 'Y', 'N')
properties_mdl['hasFireplace'] = np.where((properties_mdl['fireplaces'] == 0) | (properties_mdl['fireplaces'].isna()), 'N', 'Y')
properties_mdl['hasGarage'] = np.where((properties_mdl['garage_type'] == 0) | (properties_mdl['garage_type'].isna()), 'N', 'Y')
properties_mdl['stories'] = np.where(properties_mdl['number_stories'] == 1, 'single', np.where(properties_mdl['number_stories'] == 2, 'double', 'multiple'))
properties_mdl['area'] = np.where(properties_mdl['total_livable_area'] > properties_mdl['total_area'], properties_mdl['total_livable_area'], properties_mdl['total_area'])
properties_mdl['hasHeater'] = np.where((properties_mdl['type_heater'] == 0) | (properties_mdl['type_heater'].isna()), 'N', 'Y')
properties_mdl['quality'] = np.where(properties_mdl['quality_grade'].isin(['4', '5', '6', 'A', 'A+', 'A-', 'B', 'B+', 'B-', 'S', 'S+', 'X-']), 'Good', 'Bad')
properties_mdl['logarea'] = np.log(properties_mdl['area'])

In [ ]:
# recode building conditions
condition1 = properties_mdl['building_code_description_new'].str.contains('ROW', case=False).fillna(False).values
condition2 = properties_mdl['building_code_description_new'].str.contains('TWIN', case=False).fillna(False).values
properties_mdl['buildingdis'] = np.select(
    [condition1,condition2],
    ['Row','TWIN'],
    default='Other')

In addition to these transformations, we excluded unreasonable outliers in the training data set. We selected houses where age is less than 500 years, room count is less than 30, and both total house area and livable house area are non-zero or missing. We also excluded houses with area larger than 50,000 square feet because these are not typical single family houses. We then limited the housing prices in the training data set to be below $2,000,000 to reduce the influence of extreme values on the model.



In [7]:
# delete extreme values
properties_mdl = properties_mdl[
    (properties_mdl['Age'] < 500) &
    (properties_mdl['sale_price'] < 2000000) &
    (properties_mdl['sale_price'] > 10000) &  # Include the condition for sale_price
    (properties_mdl['numRooms'] < 30) &
    (properties_mdl['total_livable_area'] != 0) &
    (~properties_mdl['total_area'].isna()) &
    (properties_mdl['area'] < 50000)]

In [8]:
# check the original data size
properties.shape[0]

582957

In [43]:
# check the processed data size
properties_mdl.shape[0]

356342

## OLS Regression


In [ ]:
# construct dependent and independent variables
X = properties_mdl[['Age', 'numRooms', 'hasBasement', 'hasAC', 'quality', 'buildingdis', 'hasFireplace', 'hasGarage', 'stories', 'logarea', 'view', 'zip_code']]
y = properties_mdl['sale_price']
X['zip_code'] = X['zip_code'].astype(str)
X = X.dropna(subset=['zip_code'])


In [10]:
# one hot encoded dummy variables
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded

,Age,numRooms,logarea,hasBasement_Y,hasAC_Y,quality_Good,buildingdis_Row,buildingdis_TWIN,hasFireplace_Y,hasGarage_Y,...,census_tract_9.0,census_tract_90.0,census_tract_91.0,census_tract_92.0,census_tract_93.0,census_tract_94.0,census_tract_95.0,census_tract_96.0,census_tract_98.0,census_tract_nan
2,74.0,0.0,7.717351,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,104.0,0.0,8.395026,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49,134.0,0.0,7.839919,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75,109.0,0.0,7.871311,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
171,74.0,0.0,8.212568,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582939,61.0,0.0,8.932080,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
582940,60.0,0.0,9.001716,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
582944,60.0,0.0,8.984694,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
582951,56.0,0.0,8.783396,True,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [77]:
# Fit the linear regression model
reg1 = sm.OLS(y, X_encoded).fit()

# Get the summary statistics
summary_reg1 = reg1.summary()

print(summary_reg1)

                                 OLS Regression Results                                
Dep. Variable:             sale_price   R-squared (uncentered):                   0.673
Model:                            OLS   Adj. R-squared (uncentered):              0.672
Method:                 Least Squares   F-statistic:                              2170.
Date:                Wed, 03 Apr 2024   Prob (F-statistic):                        0.00
Time:                        16:21:22   Log-Likelihood:                     -4.7861e+06
No. Observations:              356342   AIC:                                  9.573e+06
Df Residuals:                  356005   BIC:                                  9.577e+06
Df Model:                         337                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [76]:
missing_values = X_encoded.isnull().sum()
print(missing_values)

Age                  0
numRooms             0
logarea              0
hasBasement_Y        0
hasAC_Y              0
                    ..
census_tract_94.0    0
census_tract_95.0    0
census_tract_96.0    0
census_tract_98.0    0
census_tract_nan     0
Length: 337, dtype: int64


## Other Possible Machine Learning Approaches

In [ ]:
# try random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize the Random Forest model
rf_model = RandomForestRegressor()

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
rf_predictions = rf_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
rf_mse = mean_squared_error(y_test, rf_predictions)
print("Random Forest Mean Squared Error:", rf_mse)


In [79]:
# try lasso

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

lasso = Lasso(alpha=1.0)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [81]:
mse1 = mean_squared_error(y_test, y_pred)

In [46]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer with one neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the testing data
y_pred = model.predict(X_test_scaled)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)


Epoch 1/50
7127/7127 [==============================] - 25s 3ms/step - loss: 47436472320.0000 - val_loss: 34351159296.0000
Epoch 2/50
7127/7127 [==============================] - 19s 3ms/step - loss: 34152699904.0000 - val_loss: 33786771456.0000
Epoch 3/50
7127/7127 [==============================] - 23s 3ms/step - loss: 33729110016.0000 - val_loss: 33458165760.0000
Epoch 4/50
7127/7127 [==============================] - 19s 3ms/step - loss: 33413384192.0000 - val_loss: 33149462528.0000
Epoch 5/50
7127/7127 [==============================] - 22s 3ms/step - loss: 33111486464.0000 - val_loss: 32838850560.0000
Epoch 6/50
7127/7127 [==============================] - 21s 3ms/step - loss: 32801413120.0000 - val_loss: 32503420928.0000
Epoch 7/50
7127/7127 [==============================] - 22s 3ms/step - loss: 32468121600.0000 - val_loss: 32152971264.0000
Epoch 8/50
7127/7127 [==============================] - 21s 3ms/step - loss: 32095676416.0000 - val_loss: 31754862592.0000
Epoch 9/50
7127/

In [ ]:
# check attribtue name
properties.columns.tolist()

['objectid',
 'assessment_date',
 'basements',
 'beginning_point',
 'book_and_page',
 'building_code',
 'building_code_description',
 'category_code',
 'category_code_description',
 'census_tract',
 'central_air',
 'cross_reference',
 'date_exterior_condition',
 'depth',
 'exempt_building',
 'exempt_land',
 'exterior_condition',
 'fireplaces',
 'frontage',
 'fuel',
 'garage_spaces',
 'garage_type',
 'general_construction',
 'geographic_ward',
 'homestead_exemption',
 'house_extension',
 'house_number',
 'interior_condition',
 'location',
 'mailing_address_1',
 'mailing_address_2',
 'mailing_care_of',
 'mailing_city_state',
 'mailing_street',
 'mailing_zip',
 'market_value',
 'market_value_date',
 'number_of_bathrooms',
 'number_of_bedrooms',
 'number_of_rooms',
 'number_stories',
 'off_street_open',
 'other_building',
 'owner_1',
 'owner_2',
 'parcel_number',
 'parcel_shape',
 'quality_grade',
 'recording_date',
 'registry_number',
 'sale_date',
 'sale_price',
 'separate_utilities',
 '

In [47]:
y_test.describe()

count    7.126900e+04
mean     1.888969e+05
std      2.182241e+05
min      1.000600e+04
25%      5.600000e+04
50%      1.250000e+05
75%      2.400000e+05
max      1.992500e+06
Name: sale_price, dtype: float64

In [ ]:
# Read in census data
import requests

# Your API key
api_key = "77ba26a94bf747d61761544063606374da01af62"

# Base URL for Census API
url = "https://api.census.gov/data/2019/acs/acs5"

# Parameters for the API request
# Adjusting to fetch data for all census tracts within Philadelphia County (county code 101) in Pennsylvania (state code 42)
params = {
    "get": "B01003_001E",  # Total population
    'for': 'tract:*',  # All tracts
    'in': 'state:42 county:101',  # Within Philadelphia County, Pennsylvania
    "key": api_key
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()
    # Create a pandas DataFrame
    # The DataFrame includes the total population for each tract in Philadelphia County
    census = pd.DataFrame(data[1:], columns=data[0])
    # Optionally, convert tract and county codes to numeric and sort
    census[['state', 'county', 'tract']] = df[['state', 'county', 'tract']].apply(pd.to_numeric)
    census.sort_values(by=['tract'], inplace=True)
    print("Successful Download")
else:
    print(f"Failed to fetch data: {response.status_code}")

Successful Download
